In [2]:
import pandas as pd 
import numpy as np
from utils import *

In [16]:
data = pd.read_csv('data/combined_datasets.csv', encoding='utf-8')
data.head()

,Unnamed: 0,id,user_name,post_message,timestamp_post,num_like_post,num_comment_post,num_share_post,label
0,0,1,389c669730cb6c54314a46be785cea42,"THĂNG CẤP BẬC HÀM ĐỐI VỚI 2 CÁN BỘ, CHIẾN SỸ H...",1585945439,19477,378,173.0,0
1,1,2,775baa6d037b6d359b229a656eaeaf08,<URL>,1588939166.0,11,5,3,0
2,2,3,b9f3394d2aff86d85974f5040c401f08,TƯ VẤN MÙA THI: Cách nộp hồ sơ để trúng tuyển ...,1591405213,48,5,19.0,0
3,3,4,808e278b22ec6b96f2faf7447d10cd8e,Cơ quan Cạnh tranh và Thị trường Anh quyết địn...,1592023613,3,0,0.0,0
4,4,5,f81bdd6d8be4c5f64bb664214e47aced,Thêm 7 ca tại Quảng Nam liên quan đến hành khá...,1583737358,775,0,54.0,0


In [17]:
# fill 0 to num_like_post, num_commnet_post, num_share_post
data.num_like_post.fillna(0, inplace=True)
data.num_comment_post.fillna(0, inplace=True)
data.num_share_post.fillna(0, inplace=True)
data = data.dropna(subset=['post_message'], how='any').reset_index(drop=True)

# check again
check_like = data.num_like_post.isna().sum()
check_commnet = data.num_comment_post.isna().sum()
check_share = data.num_share_post.isna().sum()
check_post_message = data.post_message.isna().sum()

print(f'num_like_post has nan {check_like != 0}')
print(f'num_comment_post has nan {check_commnet != 0}')
print(f'num_share_post has nan {check_share != 0}')
print(f'post_message has nan {check_post_message != 0}')

num_like_post has nan False
num_comment_post has nan False
num_share_post has nan False
post_message has nan False


In [18]:
# drop id, user_name, timestamp_post columns
data = data.drop(['timestamp_post', 'Unnamed: 0'], axis=1).reset_index(drop=True)

# label encoding for user_name
from sklearn.preprocessing import LabelEncoder
labels = data.user_name.values

le = LabelEncoder()
data['user_name_labelEncoder'] = pd.Series(le.fit_transform(labels))

# drop duplicate samples has <URL>, [<URL>](<URL>)
char_drop = ['<URL>', '[<URL>](<URL>)']
index = []
for char in char_drop:
    index.extend(list(data[data.post_message == char].index))
data = data.drop(index, axis=0).reset_index(drop=True)

# view data
data.head(5)

,id,user_name,post_message,num_like_post,num_comment_post,num_share_post,label,user_name_labelEncoder
0,1,389c669730cb6c54314a46be785cea42,"THĂNG CẤP BẬC HÀM ĐỐI VỚI 2 CÁN BỘ, CHIẾN SỸ H...",19477,378,173.0,0,1032
1,3,b9f3394d2aff86d85974f5040c401f08,TƯ VẤN MÙA THI: Cách nộp hồ sơ để trúng tuyển ...,48,5,19.0,0,2804
2,4,808e278b22ec6b96f2faf7447d10cd8e,Cơ quan Cạnh tranh và Thị trường Anh quyết địn...,3,0,0.0,0,2044
3,5,f81bdd6d8be4c5f64bb664214e47aced,Thêm 7 ca tại Quảng Nam liên quan đến hành khá...,775,0,54.0,0,3575
4,6,ffc4b6bab27c40cfc48e4dc8b8a41e42,Trong giờ học Thể dụ‌c do thầy giáo Nguyễn Văn...,2,1,0,0,3701


In [19]:
user_name_freq = data['user_name'].value_counts()

 
data['user_name_freq'] = data['user_name'].map(user_name_freq)

data.head(5)

,id,user_name,post_message,num_like_post,num_comment_post,num_share_post,label,user_name_labelEncoder,user_name_freq
0,1,389c669730cb6c54314a46be785cea42,"THĂNG CẤP BẬC HÀM ĐỐI VỚI 2 CÁN BỘ, CHIẾN SỸ H...",19477,378,173.0,0,1032,58
1,3,b9f3394d2aff86d85974f5040c401f08,TƯ VẤN MÙA THI: Cách nộp hồ sơ để trúng tuyển ...,48,5,19.0,0,2804,1
2,4,808e278b22ec6b96f2faf7447d10cd8e,Cơ quan Cạnh tranh và Thị trường Anh quyết địn...,3,0,0.0,0,2044,64
3,5,f81bdd6d8be4c5f64bb664214e47aced,Thêm 7 ca tại Quảng Nam liên quan đến hành khá...,775,0,54.0,0,3575,1
4,6,ffc4b6bab27c40cfc48e4dc8b8a41e42,Trong giờ học Thể dụ‌c do thầy giáo Nguyễn Văn...,2,1,0,0,3701,1


In [20]:
for col in ['num_comment_post', 'num_like_post', 'num_share_post']:
    data = data[pd.to_numeric(data[col], errors='coerce').notnull()]

# Đổi kiểu dữ liệu các cột sang int64
for col in ['num_comment_post', 'num_like_post', 'num_share_post']:
    data[col] = pd.to_numeric(data[col]).astype('int64')

In [21]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4957 entries, 0 to 5110
Data columns (total 9 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   id                      4957 non-null   int64 
 1   user_name               4957 non-null   object
 2   post_message            4957 non-null   object
 3   num_like_post           4957 non-null   int64 
 4   num_comment_post        4957 non-null   int64 
 5   num_share_post          4957 non-null   int64 
 6   label                   4957 non-null   int64 
 7   user_name_labelEncoder  4957 non-null   int32 
 8   user_name_freq          4957 non-null   int64 
dtypes: int32(1), int64(6), object(2)
memory usage: 367.9+ KB


In [44]:
data['post_message'] = data['post_message'].astype(str)

In [114]:
from emot.emo_unicode import UNICODE_EMO, EMOTICONS
import emoji
import unicodedata
import regex as re
import copy
import string
from vncorenlp import VnCoreNLP
from gensim.parsing.preprocessing import strip_non_alphanum, split_alphanum, strip_short, strip_numeric
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

def replace_email(text_str):
  re_email = r'\b[A-Z0-9._%+-]+@[A-Z0-9.-]+\.[A-Z]{2,}\b'
  match = re.findall(re_email ,text_str,re.IGNORECASE)
  if not match:
    return text_str
  return re.sub("|".join(match),"EMAIL",text_str)

def replace_link(text_str):
  re_link = r"(https?:\/\/)?([\da-z\.-]+)\.([a-z\.]{2,6})([\/\w \.-]*)"
  return re.sub(re_link, 'URL', text_str)

def replace_emoji(text_str):
  return emoji.get_emoji_regexp().sub(r' EMOJI ', text_str)
  
def convert_emojis(text):
    return emoji.demojize(text)

def convert_emotions(text):
    for emot in EMOTICONS:
        text = re.sub(u'('+emot+')', "_".join(EMOTICONS[emot].replace(",","").split()), text)
    return text

def get_stop_words(stopwords_file = "vietnamese-stopwords/vietnamese-stopwords.txt"):
    with open(stopwords_file, "r", encoding='utf8') as file_sw:
        stop_word = file_sw.read().split(sep='\n')
        # đưa về word dạng "xin chào" thành "xin_chào" tương ứng với bộ tokenize của vncorenlp
        stop_word = [word.replace(" ", "_") for word in stop_word]
    return stop_word

def get_teencode_dict(teencode_file = "dict_topic_word/teencode.txt"):
    teencode = pd.read_csv(teencode_file, sep="\t",
                            header=None, names=["teencode", "standard_word"])
    teencode_dict = teencode.set_index("teencode").to_dict()['standard_word']
    return teencode_dict


def remove_dup_char(text):
  return re.sub(r'(.)\1+', r'\1', text)

def text_preprocessing(text, annotator=annotator, stopwords=[]):
    # kwargs: annotator, stopwords
  origin_text = copy.deepcopy(text)  
  try:
    text = unicodedata.normalize("NFC", text)
    text = text.lower()

    # Chuẩn hóa các từ có kí tự bị lặp lại
    text = remove_dup_char(text)
    
    text = replace_email(text)
    convert_emojis(text)

    # Thay thế tất cả các đường dẫn trong văn bản.
    text = replace_link(text)
    
    #remove punctuation
    table = str.maketrans('', '',string.punctuation)
    text = text.translate(table)

    # Tách mix từ và số
    text = split_alphanum(text)
    # Loại bỏ toàn bộ các ký tự đứng 1 mình
    text = strip_short(text, minsize=2)
    # Loại bỏ hết các số trong văn bản
    text = strip_numeric(text)

    # Loại bỏ các kí tự lạ như ≥
    

    text = text.strip()
    if not text:
      return "EMPTY_STRING"
    list_token = annotator.tokenize(text)[0]

    # Chuẩn hóa teencode
    #list_token = [teencode_dict.get(token, token) for token in list_token]

    # remove stopword
    list_token = [token for token in list_token if token not in stopwords]

    norm_text = " ".join(list_token)

    return norm_text
  except:
    return "EXCEPTION_STRING"

stopwords_file = "vietnamese-stopwords/vietnamese-stopwords.txt"

stopwords = get_stop_words(stopwords_file)

annotator = VnCoreNLP("VnCoreNLP/VnCoreNLP-1.1.1.jar", annotators="wseg", max_heap_size='-Xmx500m')

data['post_message_preproced'] = data.post_message.apply(text_preprocessing, stopwords= stopwords, annotator= annotator)


In [115]:
pd.set_option('display.max_colwidth', None) 
print(data['post_message_preproced'].head(30))

0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         nhân_sĩ tâm_huyết cuồng đòi cấm xuất_khẩu gạo trung_quốc tức_giận lắm công_thương kiến_nghị gử

In [116]:
size = int(0.8*len(data))
ids_train = np.random.choice(len(data), replace=False, size=size) # ids of train
train = data.iloc[ids_train]

ids_test = [x not in ids_train for x in range(len(data))]
test = data.iloc[ids_test]

print('Size of train set:', len(train))
print('Size of test set:', len(test))

Size of train set: 3172
Size of test set: 793


In [117]:
train.to_csv('train.csv', encoding='utf8', index=False)
test.to_csv('test.csv', encoding='utf8', index=False)


In [32]:
data = pd.read_csv('train.csv', encoding='utf8')
index_0 = list(data[data.label == 0].index)
index_1 = list(data[data.label == 1].index)

ids = list(np.random.choice(index_0, size=int(1.1*len(index_1)), replace=False))

index = ids + index_1

data.iloc[index]

,id,user_name,post_message,num_like_post,num_comment_post,num_share_post,label,user_name_labelEncoder,user_name_freq,post_message_preproced
1729,1396,f3b5ef251fb740cd22686dc7b5ca0149,"Những sai phạm, vi phạm tố tụng nghiêm trọng c...",43,0,8,0,3524,3,sai_phạm vi_phạm tố_tụng nghiêm_trọng cơ_quan ...
1064,658,7232123072953674752,ĐÃ QUÁ SỨC TƯỞNG TƯỢNG CỦA LOÀI NGƯỜI - HÔM NA...,1200,522,4500,0,1834,1,quá_sức tưởng_tượng loài_người hôm_nay ca nhiễ...
761,354,6570265313371844608,"[English below] Có thể ngày mai, ngày kia, Việ...",11000,371,3600,0,1655,16,english below ngày_mai ngày_kia việt_nam ca nh...
1051,701,2184427389792466944,"Sau mấy tháng nghỉ tránh dịch COVID-19, học si...",9,2,4,0,720,7,mấy nghỉ dịch covid học_sinh hớn_hở trở_lại tr...
12,400,fa546d873f1273950bbf97bcd8fd3c0d,Từ Trạm Vũ trụ quốc tế (ISS) về Trái Đất thì c...,7,0,0,0,3614,1,trạm vũ_trụ quốc_tế is trái_đất cách_ly
...,...,...,...,...,...,...,...,...,...,...
3148,184,-4206251398616659968,"VN sẽ có khoảng 50,000 người chết trong vòng 2...",360,273,504,1,80,1,vn chết vòng nạn dịch nguy_hiểm corona số_liệu...
3151,3436,834412360cc0d5e6be6467064cd59624,Xàm 1 chút: 1 thanh niên trung quốc ở nhà trá...,7,1,1,1,2091,1,xàm chút thanh_niên trung_quốc dịch ta tách hạ...
3156,1986,2b6fe39c009f79b7e5587d9948e72621,Chịch quỵt\r\n PCI năm nay đặt ra một câu hỏi ...,2,1,0,1,844,1,chịch quỵt pci câu doanh_nghiệp công_việc kết_...
3159,2048,561bf49511b1a947aa1d801498e6efec,"Ngày 29/02/2020, chuyên gia ghép phổi nổi tiến...",38,4,0,1,1441,1,chuyên_gia ghép phổi nổi_tiếng trung_quốc trần...


In [37]:
data.iloc[index].to_csv('downsample_data.csv', encoding='utf-8', index=False)
